In [1]:
import itertools
import json
import os
from datetime import timedelta
import time
from sklearn.metrics import accuracy_score
import pandas as pd
import cv2

# My modules
from types import SimpleNamespace
from utils import set_seeds
from config import Configuration
from train_manager import TrainManager
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [2]:
def plot_confusion_matrix(cm, class_names):
    """
    https://www.tensorflow.org/tensorboard/image_summaries#building_an_image_classifier
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Compute the labels from the normalized confusion matrix.
    labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

In [3]:
def run_inference(experiment_name, tta, weight_avg, kaggle):
    config = Configuration()
    experiment_dir = os.path.abspath(f'trained-models/{experiment_name}')

    if not kaggle: 
        with open(experiment_dir + '/experiment_config.json', 'r') as f:
            config = json.load(f, object_hook=lambda d: SimpleNamespace(**d))
            set_seeds(config.seed)
        df = pd.read_csv(experiment_dir + '/holdout.csv', engine='python')
    else: 
        df = pd.DataFrame()
        df['image_id'] = list(os.listdir(config.test_img_dir))
    
    if config.num_workers > 0:
        cv2.setNumThreads(0)

    inference_start = time.time()
    print(config.model_arch)
    # get predictions... folds = None just means ensemble inference
    manager = TrainManager(folds_df=None, holdout_df=df, config=config,
                           experiment_dir=experiment_dir, experiment_name=experiment_name, kaggle=kaggle,
                          finetune=False, freeze_bn=False, freeze_feature_extractor=False)
    manager.test(tta, weight_avg, mode='vote')
    

    print(f"Inference time: {str(timedelta(seconds=time.time() - inference_start))}")
    if not kaggle:
        acc = accuracy_score(y_true=df.label.values, y_pred=manager.final_test_predictions)
        print("Ensemble holdout accuracy", acc)
        plot_confusion_matrix(manager.test_confusion_matrix.detach().cpu().numpy(), class_names=[i for i in range(config.num_classes)])
        return acc

    if kaggle:
         # make submission file
        submission = pd.DataFrame()
        submission['image_id'] = df['image_id']
        submission['label'] = manager.test_predictions_ensembled
        submission.to_csv('submission.csv', index=False)

In [ ]:
run_inference(experiment_name='tf_efficientnet_b3a_ns_sgd_1cycle_fcnodes512_smoothing=0.05_weighted_headonly2',
              tta=0,weight_avg=False, kaggle=False)

efficientnet_b3a


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


EfficientNet(
  (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
        (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
        )
        (conv_pw): Conv2d(40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
      (1): DepthwiseS

normal inference on model 0
Testing:  16%|█▌        | 8/51 [00:06<00:19,  2.16it/s]CLASSIFIER INPUT WAS NAN
 tensor([[ 2.1887e-01,  1.7273e-02,  3.3154e-01,  ...,  5.0000e-01,
          3.5449e-01,  1.6006e-02],
        [ 8.9539e-02,  3.9711e-03, -5.6641e-02,  ..., -3.1647e-02,
         -1.1340e-01, -8.3374e-02],
        [ 2.6001e-01,  8.6060e-02,  8.4229e-02,  ...,  2.7908e-02,
          3.3154e-01,  2.9248e-01],
        ...,
        [ 3.0371e-01,  1.5495e-02,  3.0655e-02,  ..., -4.6234e-02,
          4.5959e-02,  6.1426e-01],
        [ 4.4084e-04,  6.4880e-02, -6.8787e-02,  ...,  1.5796e-01,
         -3.7720e-02,  1.1261e-01],
        [-3.5797e-02, -3.3386e-02, -2.7740e-02,  ...,  1.0138e-01,
         -3.0090e-02,  2.7441e-01]], device='cuda:0', dtype=torch.float16)
classifier in min tensor(nan, device='cuda:0', dtype=torch.float16)
classifier in max tensor(nan, device='cuda:0', dtype=torch.float16)
FC1 WAS NAN
 tensor([[ 2.1887e-01,  1.7273e-02,  3.3154e-01,  ...,  5.0000e-01,
     